# Facial recognition on video clips

In [ ]:
# import the libraries
import face_recognition
import cv2
import numpy as np
import os
from os.path import isfile, join
import re

In [ ]:
# Open the input video file
input_video = cv2.VideoCapture('./data/trump2.mp4')

In [ ]:
# Store all frames as images
n = 0

while(input_video.isOpened()):
    ret, frame = input_video.read()
    
    if ret == False:
        break
    
    cv2.imwrite('./frames/frame' + str(n) + '.jpg', frame)
    n += 1

In [ ]:
# Load images
pathIn = './frames/'

frame_array = []
files = [f for f in os.listdir(pathIn)]

files.sort(key = lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

files = files[1:]

for i in range(len(files)):
    filename = pathIn + files[i]
    
    # Reading each files
    img = cv2.imread(filename)
    
    # Inserting the frames into an image array
    frame_array.append(img)

In [ ]:
# Encode trump's face
trump_image_1 = cv2.imread("./data/donald_trump.jpg")
trump_image_2 = cv2.imread("./data/donald_trump_2.jpg")
trump_image_3 = cv2.imread("./data/donald_trump_3.jpg")

trump_encoding_1 = face_recognition.face_encodings(trump_image_1)[0]
trump_encoding_2 = face_recognition.face_encodings(trump_image_2)[0]
trump_encoding_3 = face_recognition.face_encodings(trump_image_3)[0]

known_face_encodings = [trump_encoding_1, 
                        trump_encoding_2, 
                        trump_encoding_3]

In [ ]:
i = 0

#Loop over all images

for image in frame_array:
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    
    # Matches
    face_names = []

    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = None
        if matches[0]:
            name = "Donald Trump"

        face_names.append(name)

    for (top, right, bottom, left), name in zip(face_locations, face_names):
            if not name:
                continue

            # Draw a box around the face
            cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 0), 1)

            # Draw a label with a name below the face
            cv2.rectangle(image, (left, bottom + 30), (right, bottom), (0, 0, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(image, name, (left + 30, bottom + 20), font, 0.8, (255, 255, 255), 1)
    
    # Save result images
    i+=1        
    cv2.imwrite('./frames_rec/frame' + str(i) + '.jpg',image)

In [ ]:
# Load images with face rec
path = './frames_rec/'

frame_array2 = []
files2 = [f for f in os.listdir(path)]

files2.sort(key = lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

for i in range(len(files2)):
    filename = path + files2[i]
    
    #reading each files
    img = cv2.imread(filename)
    
    #inserting the frames into an image array
    frame_array2.append(img)

In [ ]:
fps = input_video.get(cv2.CAP_PROP_FPS)

width  = int(input_video.get(3))
height = int(input_video.get(4))

out = cv2.VideoWriter('./result/where_is_trump2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))

for i in range(len(frame_array)):
    out.write(frame_array[i])

input_video.release()
out.release()